## **Traveling Sales Person (TSP) Optimization with Constraint Satisfaction**

The following notebook provides a method for solving the Traveling Sales Person Problem using MiniZinc for constraint satisfaction/optimization which I developed for my class on Artificial Intelligence at the Free University of Bolzano/Bozen.

In [2]:
from minizinc import Instance, Model, Solver, Result
import nographs as nog
from scipy.sparse import lil_matrix

In [ ]:
## Enviroment ##
name: ailab-cop
channels:
  - conda-forge
  - nodefaults
dependencies:
  - python >3.8,<3.12
  - clingo
  - ipywidgets
  - jupyterlab
  - jupyterlab-git
  - jupytext
  - matplotlib
  - networkx
  - pandas
  - pip
  - pip:
      - iminizinc
      - minizinc
      - nographs
      - z3-solver

## **Maze Converter**

The following function converts a maze string into a matrix of weights for each edge, calculated by using the NoGraphs maze converter and traversal between all pairs of nodes in the graph. This information is used later by the MiniZinc solver in calculating the minimal distance.

In [17]:
def mazeconverter(mazestring):
    mazestring = mazestring.strip().splitlines() #defines the maze and splits each line to create a list of strings
    maze_arr = nog.Array(mazestring, 2) #transforms the list into a two dimensional array

    goals = (maze_arr.findall('G')) #calculates all the node coordinates
    traversal = nog.TraversalBreadthFirst(maze_arr.next_vertices_from_forbidden('#')) #defines the type of traversal used by nographs (here breadth-first)

    all_edges = []
    for num in range(len(goals)): #finds the edges and weights between each set of nodes in the whole maze and stores them in a list as a tuple of the form ((start),(goal), edge)
        temp_list = []
        for goal in goals:
            if goal != goals[num]:
                temp_list.append(goal)
        tupl = tuple(temp_list)
        infolist_g = [(traversal.depth, traversal.paths[found]) for found in traversal.start_from(start_vertices = (goals[num],), build_paths = True).go_for_vertices_in(tupl)] #extracts the path sequence and the number of moves to arrive (weight)
        edges = [(item[1][0], item[1][-1], item[0]) for item in infolist_g]
        all_edges.extend(edges) 

    goal_ids = {} 
    for i, goal in enumerate(goals): #creates ids for each of the node points (for loading the distances matrix)
        goal_ids[goal] = i

    reverse_ids = {}
    for x, y in zip(goal_ids.keys(), goal_ids.values()): #reverses the ids for each of the node points (for retrieval)
        reverse_ids[y+1] = x 

    mat = lil_matrix((len(goal_ids), len(goal_ids))) #instantiates a matrix of the length of the list the nodes 

    for x in all_edges: #updates the matrix with all the edge distances for each pair of node points
        if x[0] in goal_ids:
            id_x = goal_ids[x[0]]
            if x[1] in goal_ids:
                id_y = goal_ids[x[1]]
                mat[id_x, id_y] = x[2]

    edges = [list(int(item) for item in line) for line in mat.toarray()]
    
    return edges, reverse_ids

In [18]:
mazestring = '''
################
#G........#...G#
#..G.#....#....#
#....#..G.....G#
#G...#....#...G#
################
'''
edges, reverse_ids = mazeconverter(mazestring)
edges

[[0, 17, 3, 9, 15, 3, 16],
 [17, 0, 16, 8, 2, 20, 3],
 [3, 16, 0, 8, 14, 4, 15],
 [9, 8, 8, 0, 6, 12, 7],
 [15, 2, 14, 6, 0, 18, 1],
 [3, 20, 4, 12, 18, 0, 19],
 [16, 3, 15, 7, 1, 19, 0]]

## **MiniZinc Model**

This cell houses the problem description for the MiniZinc solver, including the variables, their frameworks, and the necessary constraints.

The script takes a integer variable of the total number of nodes and creates a set from 1 to the number of nodes. It instantiates two arrays of the size total number of nodes by total number of nodes, the first of which an empty Boolean to house the paths that the solver decides to take and the second of which the edge matrix from before. There were four total constraints: (1) the solver can only leave each node one time, (2) the solver can only enter each node one time, (3) the solver cannot take subloops, and (4) the total distance must be greater than zero. The variable total_dist sums the total of all edges taken and the solver optimizes for the minimum value. 

In [21]:
solver = Solver.lookup('gecode') 
model = Model()
model.add_string(
    """
    % Parameters
    int: nn;                        % number of nodes
    set of int: NODES = 1..nn;      % creates a set of the length of total number of nodes

    array[NODES, NODES] of var 0..1: x;             % x[i, j] is 1 if the path goes from node i to node j, otherwise it is zero
    array[NODES, NODES] of var int: edges;          % loads a matrix of the edge weights between all nodes

    constraint forall(i in NODES) (sum(j in NODES) (x[i, j]) = 1);                      % forces the salesperson to leave the goal only once, i.e. not twice or zero times
    constraint forall(j in NODES) (sum(i in NODES) (x[i, j]) = 1);                      % forces the salesperson to enter the goal only once, i.e. not twice or zero times
    constraint forall(i in NODES, j in NODES where i != j) (x[i, j] + x[j, i] <= 1);    % ensures that the salesperson completes the full loop without creating smaller loops between node point destinations 
    constraint forall(i in NODES, j in NODES where edges[i, j] == 0) (x[i, j] = 0);     % forces the solver to avoid selecting edges with zero distance (ones that indicate the same city)

    var int: total_dist = sum(i in NODES, j in NODES) (edges[i, j] * x[i, j]);          % calculates the total distance by summing the paths taken by the solver

    solve minimize total_dist; 

    output ["Nodes visited: " ++ show([i | j in NODES, i in NODES where x[i, j] = 1])];
    """
)

## **Solving for Minimal Distance, i.e. Optimal Path**

In [28]:
instance = Instance(solver, model) #creates a solver instance, loaded with the model

instance["nn"] = 7 #loading the variables into the solver instance
instance["edges"] = edges #loading the variables into the solver instance

result = (await instance.solve_async()) #solves the problem based on the constraint
try:
    optimal_path = result.solution._output_item

    hold = []
    for num in optimal_path: #outputs the optimal path, if available
        try:
            id = int(num)
            if id in reverse_ids: #reconverts the matrix coordinates into the respective node coordinates
                hold.append(reverse_ids[id])
        except:
            pass
    print(f"The Optimal Path for the TSP is: {hold}. \n") #prints the optimal sequence of nodes
    for line in result.solution.x:
        print(line) #prints the specific nodes and weights chosen from the provided edges matrix (see above)

except AttributeError:
    print('No Solution Found')

print(f"\nThe total cost of the path amounts to {result.solution.objective} spaces.") #prints the total number of spaces traversed by the solver (total weights)

The Optimal Path for the TSP is: [(2, 3), (4, 14), (4, 1), (1, 14), (3, 8), (1, 1), (3, 14)]. 

[0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0]

The total cost of the path amounts to 28 spaces.
